In [2]:
import requests
from wrapped_auth import WhoopAPI

# Create an instance of WhoopAPI
client_id = '86693dab-f7c4-448e-8fce-af6b4e5b929d'
client_secret = '0440cc1f0a10a2ef9eefdeeb7f2c1deaa489ec57aa9494a38c1826252bddda7f'
redirect_uri = 'http://localhost:8000/redirect.html'
all_scopes = ["read:profile", "read:recovery", "read:workout", "read:sleep", "read:body_measurement", "read:cycles"]

whoop_api = WhoopAPI(client_id, client_secret, redirect_uri, all_scopes)

# Step 1: Get the authorization URL and prompt user to open it in a browser
authorization_url = whoop_api.get_authorization_url()
print(f"Please go to this URL and authorize the app: {authorization_url}")

# Step 2: Get the authorization response from the user
authorization_response = input('Enter the full callback URL: ')

# Step 3: Get the access token using the authorization response
whoop_api.get_access_token(authorization_response)

# Set up headers for authorization
headers = {
    'Authorization': f'Bearer {whoop_api.access_token}'
}

Please go to this URL and authorize the app: https://api.prod.whoop.com/oauth/oauth2/auth?response_type=code&client_id=86693dab-f7c4-448e-8fce-af6b4e5b929d&redirect_uri=http%3A%2F%2Flocalhost%3A8000%2Fredirect.html&scope=read%3Aprofile+read%3Arecovery+read%3Aworkout+read%3Asleep+read%3Abody_measurement+read%3Acycles&state=SsGwTXHUZSHL4lEFrczwCS2DtSyH4D


Enter the full callback URL:  http://localhost:8000/redirect.html?code=5FtZViLuG_0UVDBbkbG9R-Fs3uO7KLRdI9-gw0BMmAo.ONY2lu31GV48vdUZsRADbiWltIxA5VF6HMlDCsUmAm4&scope=read%3Aprofile%20read%3Arecovery%20read%3Aworkout%20read%3Asleep%20read%3Abody_measurement%20read%3Acycles&state=SsGwTXHUZSHL4lEFrczwCS2DtSyH4D


In [59]:
import openai 
import json 

import os
from dotenv import load_dotenv

# Provide the path to the .env file
env_path = '/Users/galampley/Documents/api_keys.env'  # Replace this with the actual path to your .env file

# Load environment variables from .env file
load_dotenv(dotenv_path=env_path)

# Access environment variables
openai.api_key = os.getenv('OPENAI_API_KEY')

In [60]:
def get_sleep(id):
    sleep_endpoint_url = 'https://api.prod.whoop.com/developer/v1/activity/sleep'
    response = requests.get(sleep_endpoint_url, headers=headers)
    return response


# openai function calling

In [61]:
functions = [
    {
        "name": "get_sleep",
        "description": "Fetch sleep data from Whoop API by id",
        "parameters": {
            "type": "object",
            "properties": {
                "id": {
                    "type": "integer",
                    "description": "The id for which to fetch sleep data",
                },
            },
            "required": ["id"],
        },
    }
]


# llm pass 1: use llm to call the function

In [121]:
# Make an API call to OpenAI
completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "system", "content": "You are a helpful assistent to query Whoop data. Don't make any answers up. If relevant data is not in the context then say something similar to 'Not in context'"},
        {"role": "user", "content": "In records, what date is associated with sleep id 10?"},
    ],
    functions=functions,
    function_call="auto"
)

In [122]:
output = completion["choices"][0]["message"]
print(output)

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_sleep",
    "arguments": "{\n  \"id\": 10\n}"
  }
}


In [123]:
identification = json.loads(output.function_call.arguments).get("id")
params = json.loads(output.function_call.arguments)
print(identification)
print(params)

10
{'id': 10}


In [124]:
chosen_function = eval(output.function_call.name)

## api call for last 10 entries
- llm will hallucinate if it doesn't find queried data here, what if I tell system to not make things up? Kinda works

In [125]:
sleep_id = chosen_function(**params)
print(sleep_id.json())

{'records': [{'id': 781257341, 'user_id': 7036784, 'created_at': '2023-09-22T10:39:00.978Z', 'updated_at': '2023-09-22T13:47:19.278Z', 'start': '2023-09-22T05:53:24.852Z', 'end': '2023-09-22T13:29:40.090Z', 'timezone_offset': '-07:00', 'nap': False, 'score_state': 'SCORED', 'score': {'stage_summary': {'total_in_bed_time_milli': 27374277, 'total_awake_time_milli': 4671941, 'total_no_data_time_milli': 0, 'total_light_sleep_time_milli': 10649375, 'total_slow_wave_sleep_time_milli': 4902727, 'total_rem_sleep_time_milli': 7150234, 'sleep_cycle_count': 6, 'disturbance_count': 22}, 'sleep_needed': {'baseline_milli': 27887569, 'need_from_sleep_debt_milli': 4940512, 'need_from_recent_strain_milli': 1460085, 'need_from_recent_nap_milli': 0}, 'respiratory_rate': 17.578125, 'sleep_performance_percentage': 66.0, 'sleep_consistency_percentage': 89.0, 'sleep_efficiency_percentage': 85.36119}}, {'id': 780520172, 'user_id': 7036784, 'created_at': '2023-09-21T13:40:26.533Z', 'updated_at': '2023-09-21T13

# llm pass 2: add original function output to llm pass 2

In [126]:
sleep_id_json = json.dumps(sleep_id.json())

second_completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "system", "content": "You are a helpful assistent to query Whoop data. Don't make any answers up. If relevant data is not in {sleep_id_json} then say something similar to 'Not in context'"},
        {"role": "user", "content": "In records, what date is associated with sleep id 10?"},
        {"role": "function", "name": output.function_call.name, "content": sleep_id_json},
    ],
    functions=functions,
    function_call="auto"
)
response = second_completion.choices[0].message.content
print(response)

The date associated with sleep id 10 is not in context.


# pagination + all sleep data points

In [141]:
def get_sleep_v2(id=None, user_id=None, created_at=None, updated_at=None, start=None, end=None, timezone_offset=None, nap=None, score_state=None, score=None):
    sleep_endpoint_url = 'https://api.prod.whoop.com/developer/v1/activity/sleep'
    all_records = []
    next_token = None

    while True:
        # Create a dictionary for query parameters
        params = {
            'id': id,
            'user_id': user_id,
            'created_at': created_at,
            'updated_at': updated_at,
            'start': start,
            'end': end,
            'timezone_offset': timezone_offset,
            'nap': nap,
            'score_state': score_state,
            'score': score
        }

        if next_token:
            params["nextToken"] = next_token

        # Remove None values from params
        params = {k: v for k, v in params.items() if v is not None}

        response = requests.get(sleep_endpoint_url, headers=headers, params=params)
        
        all_data = response.json()

        all_records.extend(all_data.get("records", []))
        next_token = all_data.get("next_token")

        if not next_token:
            break

    return all_records


In [142]:
functions_v2 = [
    {
        "name": "get_sleep_v2",
        "description": "Get sleep data for a single day or a range of days. If relevant, calculate average, max or min of sleep data.",
        "parameters": {
            "type": "object",
            "properties": {
                "id": {"type": "integer", "description": "Unique identifier for the sleep activity."},
                "user_id": {"type": "integer", "description": "The WHOOP User who performed the sleep activity."},
                "created_at": {"type": "string", "description": "The time the sleep activity was recorded in WHOOP."},
                "updated_at": {"type": "string", "description": "The time the sleep activity was last updated in WHOOP."},
                "start": {"type": "string", "description": "Start time bound of the sleep. Represented as ISO 8601 timestamp."},
                "end": {"type": "string", "description": "End time bound of the sleep. Represented as ISO 8601 timestamp"},
                "timezone_offset": {"type": "string", "description": "The user's timezone offset at the time the sleep was recorded."},
                "nap": {"type": "boolean", "description": "If true, this sleep activity was a nap for the user."},
                "score_state": {
                    "type": "string",
                    "description": "The scoring state of the sleep activity.",
                    "enum": ["SCORED", "PENDING_SCORE", "UNSCORABLE"]
                },
                "score": {
                    "type": "object",
                    "description": "WHOOP's measurements and evaluation of the sleep activity.",
                    "properties": {
                        "stage_summary": {
                            "type": "object",
                            "description": "Summary of different sleep stages.",
                            "properties": {
                                "total_in_bed_time_milli": {"type": "integer", "description": "Total time in bed in milliseconds."},
                                "total_awake_time_milli": {"type": "integer", "description": "Total awake time in milliseconds."},
                                "total_no_data_time_milli": {"type": "integer", "description": "Total time with no data in milliseconds."},
                                "total_light_sleep_time_milli": {"type": "integer", "description": "Total light sleep time in milliseconds."},
                                "total_slow_wave_sleep_time_milli": {"type": "integer", "description": "Total slow-wave sleep time in milliseconds."},
                                "total_rem_sleep_time_milli": {"type": "integer", "description": "Total REM sleep time in milliseconds."},
                                "sleep_cycle_count": {"type": "integer", "description": "Total number of sleep cycles."},
                                "disturbance_count": {"type": "integer", "description": "Total number of disturbances during sleep."}
                            },
                        },
                        "sleep_needed": {
                            "type": "object",
                            "description": "Sleep needed calculated from various factors.",
                            "properties": {
                                "baseline_milli": {"type": "integer", "description": "Baseline sleep needed in milliseconds."},
                                "need_from_sleep_debt_milli": {"type": "integer", "description": "Additional sleep needed due to sleep debt, in milliseconds."},
                                "need_from_recent_strain_milli": {"type": "integer", "description": "Additional sleep needed due to recent strain, in milliseconds."},
                                "need_from_recent_nap_milli": {"type": "integer", "description": "Sleep needed adjustment due to recent nap, in milliseconds."}
                            },
                        },
                        "respiratory_rate": {"type": "number", "description": "Rate of breathing while asleep."},
                        "sleep_performance_percentage": {"type": "number", "description": "How well the Whoop User slept."},
                        "sleep_consistency_percentage": {"type": "number", "description": "How consistent the Whoop User slept."},
                        "sleep_efficiency_percentage": {"type": "number", "description": "How efficient the Whoop User slept."}
                    },
                },
            },
        },
    }
]


In [143]:
user_query = "What was my baseline sleep needed, from 8/28/23 to 9/2/23?"

In [144]:
# Make an API call to OpenAI
completion_v2 = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": user_query}
    ],
    functions=functions_v2,
    function_call="auto"
)

In [145]:
output_v2 = completion_v2["choices"][0]["message"]
print(output_v2)

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_sleep_v2",
    "arguments": "{\n  \"start\": \"2023-08-28T00:00:00Z\",\n  \"end\": \"2023-09-02T23:59:59Z\"\n}"
  }
}


In [146]:
params_v2 = json.loads(output_v2.function_call.arguments)
print(params_v2)

{'start': '2023-08-28T00:00:00Z', 'end': '2023-09-02T23:59:59Z'}


In [147]:
chosen_function_v2 = eval(output_v2.function_call.name)

In [148]:
sleep_data = chosen_function_v2(**params_v2)
print(sleep_data)

[{'id': 762638632, 'user_id': 7036784, 'created_at': '2023-09-02T13:46:26.897Z', 'updated_at': '2023-09-02T13:46:32.028Z', 'start': '2023-09-02T05:54:10.409Z', 'end': '2023-09-02T13:27:59.541Z', 'timezone_offset': '-07:00', 'nap': False, 'score_state': 'SCORED', 'score': {'stage_summary': {'total_in_bed_time_milli': 27228171, 'total_awake_time_milli': 3950965, 'total_no_data_time_milli': 0, 'total_light_sleep_time_milli': 11361562, 'total_slow_wave_sleep_time_milli': 6263134, 'total_rem_sleep_time_milli': 5652510, 'sleep_cycle_count': 7, 'disturbance_count': 11}, 'sleep_needed': {'baseline_milli': 27893554, 'need_from_sleep_debt_milli': 2331104, 'need_from_recent_strain_milli': 2505425, 'need_from_recent_nap_milli': 0}, 'respiratory_rate': 18.046875, 'sleep_performance_percentage': 71.0, 'sleep_consistency_percentage': 84.0, 'sleep_efficiency_percentage': 85.489426}}, {'id': 761469097, 'user_id': 7036784, 'created_at': '2023-09-01T09:12:07.656Z', 'updated_at': '2023-09-01T14:39:09.298Z

In [149]:
sleep_data_json = json.dumps(sleep_data)

second_completion_v2 = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[  
       {"role": "system", "content": "Always convert milliseconds to hours. There are 3.6 million millseconds in an hour."},
        {"role": "user", "content": user_query},
        {"role": "function", "name": output_v2.function_call.name, "content": sleep_data_json},
    ],
    functions=functions_v2,
)
response_v2 = second_completion_v2.choices[0].message.content
print(response_v2)

The baseline sleep needed from 8/28/23 to 9/2/23 was as follows:

- 8/28/23: 27895055 milliseconds (7 hours 44 minutes)
- 8/29/23: 27894755 milliseconds (7 hours 44 minutes)
- 8/30/23: 27894454 milliseconds (7 hours 44 minutes)
- 8/31/23: 27894154 milliseconds (7 hours 44 minutes)
- 9/1/23: 27893854 milliseconds (7 hours 44 minutes)
- 9/2/23: 27893554 milliseconds (7 hours 44 minutes)

Please note that these values are given in milliseconds.
